# Monitoring SNOWFLAKE Table Usage - Analysis ❄️

# Part1: Snowflake Worksheet SQL ⬇️

### Select Warehouse for Analysis

### Download Query_Count Data

### Download Active_GB Table

### Download ROW_COUNT Table

# Part2: Python Analysis | Table Preparation 🐍

## Importing Libraries and Tables

In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import re
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
warnings.filterwarnings('ignore')

#Importing the necessary CSV's from SNOWFLAKE Worksheet.

qCountTable = pd.read_csv('Location of your qcount csv')
activeGB = pd.read_csv('Location of your table size csv')
rowCount = pd.read_csv('Location of your row count csv')

## Query Count

In [2]:
qCountTable = qCountTable.rename(columns = {'T0.VALUE:""OBJECTNAME""':'Tables'})
qCountTable.head()

,QUERY_START_TIME,Tables
0,2021-11-15 13:47:18.648 -0800,"INSPIRE_PDB.PDB.COPY_TEST_ORDER_DETAILS"""""
1,2021-11-15 11:56:24.189 -0800,"INSPIRE_PDB.PDB.COPY_TEST_ORDER_DETAILS"""""
2,2021-11-15 12:17:49.645 -0800,"INSPIRE_PDB.PDB.JAN_DM_ECL_SELECTED_ID_20201115"""""
3,2021-11-15 12:17:49.645 -0800,"INSPIRE_PDB.PDB.SELECTED_11_0"""""
4,2021-11-15 11:26:04.531 -0800,INSPIRE_PDB.PDB.INSPIRE_PDB_MARKETING_UNIVERSE...


In [3]:
#Converting to datetime and localising timezone.
qCountTable['QUERY_START_TIME'] = pd.to_datetime(qCountTable['QUERY_START_TIME'], utc=True)
qCountTable['QUERY_START_TIME'] = qCountTable['QUERY_START_TIME'].dt.tz_localize(None)
qCountTable.head(1)

,QUERY_START_TIME,Tables
0,2021-11-15 21:47:18.648,"INSPIRE_PDB.PDB.COPY_TEST_ORDER_DETAILS"""""


In [4]:
#Cleaning table name to remove qoutation marks.
qCountTable['Tables'] = qCountTable['Tables'].apply(lambda x: re.sub(r'[""]*', '', str(x)))
qCountTable.sort_values(by=['QUERY_START_TIME'])

,QUERY_START_TIME,Tables
28865,2021-08-31 20:07:06.339,C2G_LAKE_DEV.VDS_FTP.INS_JULYCAMP_DM_TO_INS_OR...
13378,2021-08-31 20:07:19.007,C2G_LAKE_DEV.VDS_FTP.INS_JULYCAMP_DM_TO_INS_OR...
3257,2021-09-01 17:17:59.213,INSPIRE_SHARE.TEST.N
13379,2021-09-01 17:29:47.694,INSPIRE_SHARE.TEST.N
18492,2021-09-01 17:30:05.635,INSPIRE_SHARE.TEST.N
...,...,...
13467,2022-03-17 13:29:00.569,INSPIRE_PDB.PDB.JAN22_MAILED
13468,2022-03-17 13:29:00.569,INSPIRE_PDB.PDB.FEB22_MAILED
32482,2022-03-17 13:29:01.529,SNOWFLAKE.ACCOUNT_USAGE.ACCESS_HISTORY$V2
24139,2022-03-17 13:38:30.583,SNOWFLAKE.ACCOUNT_USAGE.ACCESS_HISTORY$V2


In [5]:
#We want to group my month, so we need to alter datetime format to Month/Year.
qCountTable['Month/Year'] = pd.to_datetime(qCountTable['QUERY_START_TIME']).dt.strftime('%m/%Y')

In [6]:
#This groups tables that have the same name and counts their frequency for a given month/year.
qCountTable2 = qCountTable.groupby(['Month/Year', 'Tables']).size()

In [7]:
#Resetting indexes, renaming 'Count' column and dropping irrelevant query-start-time.
qCountTable2 = qCountTable2.reset_index()
qCountAgg = qCountTable.merge(qCountTable2).rename(columns={0:'Count'})
qCountAgg = qCountAgg.drop(['QUERY_START_TIME'], axis=1)
qCountAgg.head()

,Tables,Month/Year,Count
0,INSPIRE_PDB.PDB.COPY_TEST_ORDER_DETAILS,11/2021,2
1,INSPIRE_PDB.PDB.COPY_TEST_ORDER_DETAILS,11/2021,2
2,INSPIRE_PDB.PDB.JAN_DM_ECL_SELECTED_ID_20201115,11/2021,16
3,INSPIRE_PDB.PDB.JAN_DM_ECL_SELECTED_ID_20201115,11/2021,16
4,INSPIRE_PDB.PDB.JAN_DM_ECL_SELECTED_ID_20201115,11/2021,16


In [8]:
#First precaution to drop duplicates.
uniqueTableNames = list(qCountAgg['Tables'].unique())
uniqueTableNames = [str(i) for i in uniqueTableNames]
uniqueTableNames = set(uniqueTableNames)

#Not interested in access history regarding Information Schema or SNOWFLAKE Schemas.
#List comprehension creates list of table names filtering out snowflake/info schemas.
newTables = [elements for elements in uniqueTableNames if 'INFORMATION_SCHEMA' not in elements]
newTables = [elements for elements in newTables if 'SNOWFLAKE' not in elements]

In [9]:
qCountFiltered = pd.DataFrame() #Creating empty DataFrame to be populated.
qCountFiltered['Tables'] = []
qCountFiltered['Month/Year'] = []
qCountFiltered['Count'] = []

'''Creates boolean template for DF, filters DF for where the name is in filtered list.
As the table is already ordered by date and we only want the most recent,
it takes the last entry for each given name using tail.'''

for i in newTables: 
    try:
        is_table = qCountAgg['Tables']==i
        output_table = qCountAgg[is_table]
        tailed_output = output_table.tail(1)
        #tailed_value = tailed_output.values[0]
        #if tailed_value[1] not in recent_months: #NOT IN = OLD, IN = RECENT!
        qCountFiltered = pd.concat([qCountFiltered, tailed_output])
    except:
        print(i)
        pass

In [10]:
qCountFiltered.head(5)

,Tables,Month/Year,Count
37021,INSPIRE_PDB.PDB.TEMP_NONECL_ONLY_DUPS,10/2021,13.0
40482,INSPIRE_PDB.PDB.SPARK_CONNECTOR_LOAD_STAGE_M5D...,02/2022,2.0
39937,INSPIRE_PDB.PDB.SPARK_CONNECTOR_LOAD_STAGE_UI3...,02/2022,2.0
31676,INSPIRE_PDB.PDB.JAN_DM_ECL_SELECTED_ID_20201110,11/2021,16.0
40314,INSPIRE_PDB.PDB.NON_ECL_XGB_PREPROCESSING_TNI,02/2022,4.0


## Appending DB Size

In [11]:
activeGB.head(1)

,TABLE_NAME,ACTIVE_GB
0,NAAZ5,0.011123


In [12]:
qCountFiltered = qCountFiltered.reset_index()
qCountFiltered.head(1)

,index,Tables,Month/Year,Count
0,37021,INSPIRE_PDB.PDB.TEMP_NONECL_ONLY_DUPS,10/2021,13.0


In [13]:
''' The table name includes the database, schema and table. To change this,
we split on '.' and take only the table name from the chain. This is then set
as table name.'''

for index, element in enumerate(qCountFiltered['Tables']):
    try:
        name_split = element.split('.')
        qCountFiltered.at[index, 'Tables']=name_split[2]
    except:
        print(element, name_split)
        pass

nan ['nan']


In [14]:
#Renaming Columns
qCountFiltered = qCountFiltered.rename(columns={"index": "remove", "Month/Year": "LAST_QUERIED", "Tables": "TABLE_NAME", "Count": "Q_COUNT"})

In [15]:
qCountFiltered.head()

,remove,TABLE_NAME,LAST_QUERIED,Q_COUNT
0,37021,TEMP_NONECL_ONLY_DUPS,10/2021,13.0
1,40482,SPARK_CONNECTOR_LOAD_STAGE_M5DLZYBP1L,02/2022,2.0
2,39937,SPARK_CONNECTOR_LOAD_STAGE_UI3SITOS9R,02/2022,2.0
3,31676,JAN_DM_ECL_SELECTED_ID_20201110,11/2021,16.0
4,40314,NON_ECL_XGB_PREPROCESSING_TNI,02/2022,4.0


In [16]:
#Joining with DB size table on table name.
tableMerged = pd.merge(qCountFiltered,activeGB,on='TABLE_NAME')
tableMerged = tableMerged.drop(['remove'], axis=1)
tableMerged.head()

,TABLE_NAME,LAST_QUERIED,Q_COUNT,ACTIVE_GB
0,TEMP_NONECL_ONLY_DUPS,10/2021,13.0,0.033498
1,JAN_DM_ECL_SELECTED_ID_20201110,11/2021,16.0,0.032910
2,NON_ECL_XGB_PREPROCESSING_TNI,02/2022,4.0,4.371299
3,FEB22_DM_FROM_INSPIRE_ENGINEERING,03/2022,2.0,0.115091
4,FEB22_DM_FROM_INSPIRE_ENGINEERING,03/2022,4.0,0.115091


In [17]:
#Sorting by db size to prioritise most expensive tables. Round to 3d.p.
tableMerged = tableMerged.sort_values(by=['ACTIVE_GB'], ascending=False)
tableMerged = tableMerged.reset_index()
tableMerged['ACTIVE_GB'] = tableMerged['ACTIVE_GB'].round(decimals = 3)
tableMerged.head()

,index,TABLE_NAME,LAST_QUERIED,Q_COUNT,ACTIVE_GB
0,96,EPSILON_LATEST_210817,01/2022,1.0,391.781
1,123,ALTAIR_HISTORICAL_051221,10/2021,36.0,303.713
2,389,ALTAIR_LATEST,03/2022,1.0,298.478
3,406,ALTAIR_LATEST_DEV,11/2021,1.0,288.011
4,129,EPSILON_LATEST_201117,11/2021,1.0,177.336


## Appending Row Count + Additional Fields

In [18]:
rowCount.head()

,TABLE_NAME,ROW_COUNT,SCHEMA,CREATED,DATABASE
0,NLAZIP2,1001227,ENRICHMENT,2021-08-31 11:35:05.882 -0700,C2G_LAKE_DEV
1,ALTAIR_HISTORICAL_042420,172765770,VDS_FTP,2021-08-31 11:35:05.965 -0700,C2G_LAKE_DEV
2,ALTAIR_PSCS_ZIP9_210505,11786062,VDS_FTP,2021-08-31 11:35:06.095 -0700,C2G_LAKE_DEV
3,NAA_ZIP5_20210629,37889,VDS_FTP,2021-08-31 11:35:06.179 -0700,C2G_LAKE_DEV
4,ALTAIR_HISTORICAL_051221,160584356,VDS_FTP,2021-08-31 11:35:06.271 -0700,C2G_LAKE_DEV


In [19]:
''' Merge with row count table, bringing previously discarded information such as schema
and database alongside row count and the tables created date.'''

tableMerged = pd.merge(tableMerged,rowCount,on='TABLE_NAME')
tableMerged['ROW_COUNT'] = tableMerged.apply(lambda x: "{:,}".format(x['ROW_COUNT']), axis=1)
tableMerged['CREATED'] = pd.to_datetime(tableMerged['CREATED'], utc=True)
tableMerged['CREATED'] = tableMerged['CREATED'].dt.tz_localize(None)
tableMerged['CREATED'] = tableMerged['CREATED'].apply(lambda x: "%d/%d/%d" % (x.month, x.day, x.year))
tableMerged.head()

,index,TABLE_NAME,LAST_QUERIED,Q_COUNT,ACTIVE_GB,ROW_COUNT,SCHEMA,CREATED,DATABASE
0,96,EPSILON_LATEST_210817,01/2022,1.0,391.781,"726,407,889",VDS_FTP,8/31/2021,C2G_LAKE_DEV
1,123,ALTAIR_HISTORICAL_051221,10/2021,36.0,303.713,"160,584,356",VDS_FTP,8/31/2021,C2G_LAKE_DEV
2,389,ALTAIR_LATEST,03/2022,1.0,298.478,"160,808,223",VDS_FTP,10/15/2021,C2G_LAKE_DEV
3,406,ALTAIR_LATEST_DEV,11/2021,1.0,288.011,"159,380,782",VDS_FTP,8/31/2021,C2G_LAKE_DEV
4,129,EPSILON_LATEST_201117,11/2021,1.0,177.336,"363,924,955",VDS_FTP,8/31/2021,C2G_LAKE_DEV


In [20]:
#Dropping duplicates and renaming columns. - General table restructuring.
tableMerged = tableMerged.drop_duplicates()
tableMerged.reset_index(inplace = True)
tableMerged.pop('index')
tableMerged.pop('level_0')
tableMerged = tableMerged.rename(columns={"TABLE_NAME": "TABLE"})
tableMerged.head()

,TABLE,LAST_QUERIED,Q_COUNT,ACTIVE_GB,ROW_COUNT,SCHEMA,CREATED,DATABASE
0,EPSILON_LATEST_210817,01/2022,1.0,391.781,"726,407,889",VDS_FTP,8/31/2021,C2G_LAKE_DEV
1,ALTAIR_HISTORICAL_051221,10/2021,36.0,303.713,"160,584,356",VDS_FTP,8/31/2021,C2G_LAKE_DEV
2,ALTAIR_LATEST,03/2022,1.0,298.478,"160,808,223",VDS_FTP,10/15/2021,C2G_LAKE_DEV
3,ALTAIR_LATEST_DEV,11/2021,1.0,288.011,"159,380,782",VDS_FTP,8/31/2021,C2G_LAKE_DEV
4,EPSILON_LATEST_201117,11/2021,1.0,177.336,"363,924,955",VDS_FTP,8/31/2021,C2G_LAKE_DEV


In [21]:
#Re-organising column order.
column_order = ['CREATED', 'LAST_QUERIED', 'DATABASE', 'SCHEMA', 'TABLE', 'Q_COUNT', 'ROW_COUNT', 'ACTIVE_GB']
tableMerged = tableMerged[column_order]
tableMerged.head(5)
print(len(tableMerged))

437


## Dropping Duplicates

In [22]:
length1 = len(tableMerged)
finalTable = tableMerged.drop_duplicates(subset=['DATABASE', 'SCHEMA', 'TABLE'], keep = 'last')
length2 = len(finalTable)

In [23]:
duplicate_count = length1 - length2
print('Total duplicates removed =', duplicate_count)

Total duplicates removed = 69


## Separating Old Vs Recent

In [24]:
recent_months = []
today = date.today() #Calculates todays date
d1 = today.strftime("%m/%Y") #Alters date format
count = 0

while count < 3: #This function creates a list of the last 3 months.
    d2 = date.today() - relativedelta(months=+count)
    d2 = d2.strftime("%m/%Y")
    recent_months.append(d2)
    count += 1

print(recent_months)

['03/2022', '02/2022', '01/2022']


In [25]:
df_recent = pd.DataFrame(columns = ['CREATED', 'LAST_QUERIED', 'DATABASE', 'SCHEMA', 'TABLE', 'Q_COUNT', 'ROW_COUNT', 'ACTIVE_GB'])
df_old = pd.DataFrame(columns = ['CREATED', 'LAST_QUERIED', 'DATABASE', 'SCHEMA', 'TABLE', 'Q_COUNT', 'ROW_COUNT', 'ACTIVE_GB'])

for index, row in finalTable.iterrows(): 
        if row['LAST_QUERIED'] not in recent_months: #NOT IN = OLD, IN = RECENT!
            df_old = df_old.append(row)
        else:
            df_recent = df_old.append(row)

In [26]:
#Output tables to csv format.
df_recent.to_csv('Path to save recent data usage')
df_old.to_csv('Path to save old data usage')